# Transportation in California Counties 

## Preliminaries

In [1]:
# Dependencies
import pylab as plt
import pandas as pd
import numpy as np
import os
import time
from sqlalchemy import create_engine

## Extract

In [2]:
# File Paths to CSVs we are importing
traffic_modes_path = os.path.join("Resources", "transportation-to-work.csv")
traffic_injuries_path = os.path.join("Resources", "road-traffic-injuries.csv")

# Use Pandas to read CSV files
traffic_modes_df = pd.read_csv(traffic_modes_path, low_memory=False)
traffic_injuries_df = pd.read_csv(traffic_injuries_path, low_memory=False)

## Transform
### Group Datasets to Build Category Specific Tables

In [3]:
# Build reference tables orgnaized by county, region, transportation modes, and race and ethnicity
### Tables for traffic_modes_df
TM_counties_df = pd.DataFrame\
    (traffic_modes_df.groupby(['county_name', 'county_fips', 'region_code']).size())

TM_regions_df = pd.DataFrame\
    (traffic_modes_df.groupby(['region_name', 'region_code']).size())

TM_transport_modes_df = pd.DataFrame\
    (traffic_modes_df.groupby(['mode', 'mode_name']).size()) 

TM_race_eth_df = pd.DataFrame\
    (traffic_modes_df.groupby(['race_eth_name', 'race_eth_code']).size())  

### Tables for traffic_injuries_df
TI_counties_df = pd.DataFrame\
    (traffic_injuries_df.groupby(['county_name', 'county_fips', 'region_code']).size())

TI_regions_df = pd.DataFrame\
    (traffic_injuries_df.groupby(['region_name', 'region_code']).size())

TI_transport_modes_df = pd.DataFrame\
    (traffic_injuries_df.groupby(['mode']).size())                           

TI_race_eth_df = pd.DataFrame\
    (traffic_injuries_df.groupby(['race_eth_name', 'race_eth_code']).size())

In [4]:
# Delete unecessary columns and reset indices
df_list = [TM_counties_df, TM_regions_df, 
           TM_transport_modes_df, TM_race_eth_df,
           TI_counties_df, TI_regions_df, 
           TI_transport_modes_df, TI_race_eth_df]

for df in df_list:
    df.reset_index(inplace=True)
    del df[0]

In [5]:
# Join traffic_modes tables to traffic_injuries table for all categories accept transportation modes
counties_joined = pd.merge(TM_counties_df, TI_counties_df, \
                           on=['county_name', 'county_fips', 'region_code'], how='outer')

regions_joined = pd.merge(TM_regions_df, TI_regions_df, \
                          on=['region_name', 'region_code'], how='outer')

race_eth_joined = pd.merge(TM_race_eth_df, TI_race_eth_df, \
                           on=['race_eth_name', 'race_eth_code'], how='outer')

In [6]:
# We want to provide total population per county, only traffic_injuries_df supplies this info
total_pop_per_county = traffic_injuries_df\
    [['county_name','totalpop']].groupby(['county_name']).sum()

# Reset the index so that 'county_name' is treated as a column
total_pop_per_county.reset_index(inplace=True)
total_pop_per_county.head()

,county_name,totalpop
0,Alameda,4.861985e+08
1,Alpine,1.628877e+05
2,Amador,6.461525e+06
3,Butte,5.474099e+07
4,Calaveras,7.369064e+06


In [7]:
# Add totalpop to counties_joined
counties_pop_joined = pd.merge\
    (counties_joined, total_pop_per_county, on='county_name', how='outer')

# Adjust number format of 'totalpop' column
counties_pop_joined['totalpop'] = counties_pop_joined['totalpop']/(10**5)
counties_pop_joined['totalpop'] = counties_pop_joined['totalpop'].round(2)

counties_pop_joined.head()

,county_name,county_fips,region_code,totalpop
0,Alameda,6001.0,1.0,4861.99
1,Alpine,6003.0,3.0,1.63
2,Amador,6005.0,3.0,64.62
3,Butte,6007.0,2.0,547.41
4,Calaveras,6009.0,3.0,73.69


### Filter Datasets for Desired Statistics

In [8]:
# Create a table that documents workers modes of transportation
# and is linked to the counties/race_eth tables 
### We are only concerned with data corresponding to counties 
    ### so disregard rows with data corresponding to other geotypes

modes_of_transportation = traffic_modes_df\
    [['mode','mode_name', 'pop_mode','county_fips', 'race_eth_code']].\
loc[traffic_modes_df['county_fips'].isna() == False]

# Reset index so that it starts from 0
modes_of_transportation.reset_index(level=0, inplace=True); del modes_of_transportation['index']

# NAN in 'popmode' corresponds to 0 workers so replace
modes_of_transportation = modes_of_transportation.fillna(value=0)

modes_of_transportation.head()

,mode,mode_name,pop_mode,county_fips,race_eth_code
0,WALK,Walked to work,0.0,6003.0,5
1,ATHOME,Worked at home,54.0,6001.0,1
2,ATHOME,Worked at home,3001.0,6001.0,2
3,ATHOME,Worked at home,2287.0,6001.0,3
4,ATHOME,Worked at home,2080.0,6001.0,4


In [9]:
# Create a table that documents road traffic injuries
# and is linked to the counties/race_eth tables 

### We are only concerned with data corresponding to counties 
    ### so disregard rows with data corresponding to other geotypes

traffic_injuries = traffic_injuries_df\
    [['severity', 'injuries', 'poprate', 'mode', 'county_fips', 'race_eth_code']].\
loc[traffic_injuries_df['county_fips'].isna() == False]

# Reset index so that it starts from 0
traffic_injuries.reset_index(level=0, inplace=True); del traffic_injuries['index']

# NAN in 'popmode' corresponds to 0 workers so replace
traffic_injuries = traffic_injuries.fillna(value=0)
traffic_injuries.head()

,severity,injuries,poprate,mode,county_fips,race_eth_code
0,Killed,1.0,0.0,Vehicles,6057.0,9
1,Killed,7.0,0.0,All modes,6037.0,9
2,Severe Injury,44.0,0.0,All modes,6037.0,9
3,Severe Injury,1.0,0.0,Bicyclist,6037.0,9
4,Killed,3.0,0.0,Car/Pickup,6037.0,9


### Rename Tables and Columns 
- Make sure dataframe names are in accordance with SQL schema

In [10]:
counties = counties_pop_joined.rename(columns={'county_fips': 'county_fips_code',
                                               'totalpop': 'totalpop'
                                              }
                                     )
counties.head()

,county_name,county_fips_code,region_code,totalpop
0,Alameda,6001.0,1.0,4861.99
1,Alpine,6003.0,3.0,1.63
2,Amador,6005.0,3.0,64.62
3,Butte,6007.0,2.0,547.41
4,Calaveras,6009.0,3.0,73.69


In [11]:
regions = regions_joined
regions.head()

,region_name,region_code
0,Bay Area,1.0
1,Butte,2.0
2,Central/Southeast Sierra,3.0
3,Monterey Bay,4.0
4,North Coast,5.0


In [12]:
ethnicity = race_eth_joined

In [13]:
modes_of_transportation = modes_of_transportation.\
                            rename(columns ={'mode': 'transport_name',
                                             'mode_name': 'transport_description',
                                             'pop_mode': 'num_workers_per_transport',
                                             'county_fips': 'county_fips_code'
                                            }
                                  )
modes_of_transportation.head()

,transport_name,transport_description,num_workers_per_transport,county_fips_code,race_eth_code
0,WALK,Walked to work,0.0,6003.0,5
1,ATHOME,Worked at home,54.0,6001.0,1
2,ATHOME,Worked at home,3001.0,6001.0,2
3,ATHOME,Worked at home,2287.0,6001.0,3
4,ATHOME,Worked at home,2080.0,6001.0,4


In [14]:
traffic_injuries.columns
traffic_injuries = traffic_injuries.rename(columns={'poprate': 'rate_of_injuries_in_pop',
                                                    'mode': 'injury_transport_mode',
                                                    'county_fips': 'county_fips_code'
                                                   }
                                          )
traffic_injuries.head()

,severity,injuries,rate_of_injuries_in_pop,injury_transport_mode,county_fips_code,race_eth_code
0,Killed,1.0,0.0,Vehicles,6057.0,9
1,Killed,7.0,0.0,All modes,6037.0,9
2,Severe Injury,44.0,0.0,All modes,6037.0,9
3,Severe Injury,1.0,0.0,Bicyclist,6037.0,9
4,Killed,3.0,0.0,Car/Pickup,6037.0,9


### Connect to local database

In [15]:

rds_connection_string = "postgres:Ech04.red!@localhost:5432/etl_project2"
engine = create_engine(f'postgresql://{rds_connection_string}')


### Check for tables

In [16]:
engine.table_names()

['regions',
 'counties',
 'modes_of_transportation',
 'ethnicity',
 'traffic_injuries']

### Use pandas to load csv converted DataFrame into database

In [19]:
regions.to_sql(name='regions', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pk_regions"
DETAIL:  Key (region_code)=(1) already exists.

[SQL: INSERT INTO regions (region_name, region_code) VALUES (%(region_name)s, %(region_code)s)]
[parameters: ({'region_name': 'Bay Area', 'region_code': 1.0}, {'region_name': 'Butte', 'region_code': 2.0}, {'region_name': 'Central/Southeast Sierra', 'region_code': 3.0}, {'region_name': 'Monterey Bay', 'region_code': 4.0}, {'region_name': 'North Coast', 'region_code': 5.0}, {'region_name': 'Northeast Sierra', 'region_code': 6.0}, {'region_name': 'Northern Sacramento Valley', 'region_code': 7.0}, {'region_name': 'Sacramento Area', 'region_code': 8.0}  ... displaying 10 of 14 total bound parameter sets ...  {'region_name': 'Shasta', 'region_code': 13.0}, {'region_name': 'Southern California', 'region_code': 14.0})]
(Background on this error at: http://sqlalche.me/e/gkpj)

In [20]:
counties.to_sql(name='counties', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pk_counties"
DETAIL:  Key (county_fips_code)=(6001) already exists.

[SQL: INSERT INTO counties (county_name, county_fips_code, region_code, totalpop) VALUES (%(county_name)s, %(county_fips_code)s, %(region_code)s, %(totalpop)s)]
[parameters: ({'county_name': 'Alameda', 'county_fips_code': 6001.0, 'region_code': 1.0, 'totalpop': 4861.99}, {'county_name': 'Alpine', 'county_fips_code': 6003.0, 'region_code': 3.0, 'totalpop': 1.63}, {'county_name': 'Amador', 'county_fips_code': 6005.0, 'region_code': 3.0, 'totalpop': 64.62}, {'county_name': 'Butte', 'county_fips_code': 6007.0, 'region_code': 2.0, 'totalpop': 547.41}, {'county_name': 'Calaveras', 'county_fips_code': 6009.0, 'region_code': 3.0, 'totalpop': 73.69}, {'county_name': 'Colusa', 'county_fips_code': 6011.0, 'region_code': 7.0, 'totalpop': 36.04}, {'county_name': 'Contra Costa', 'county_fips_code': 6013.0, 'region_code': 1.0, 'totalpop

In [21]:
ethnicity.to_sql(name='ethnicity', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pk_ethnicity"
DETAIL:  Key (race_eth_code)=(1) already exists.

[SQL: INSERT INTO ethnicity (race_eth_name, race_eth_code) VALUES (%(race_eth_name)s, %(race_eth_code)s)]
[parameters: ({'race_eth_name': 'AIAN', 'race_eth_code': 1}, {'race_eth_name': 'AfricanAm', 'race_eth_code': 3}, {'race_eth_name': 'Asian', 'race_eth_code': 2}, {'race_eth_name': 'Latino', 'race_eth_code': 4}, {'race_eth_name': 'Multiple', 'race_eth_code': 7}, {'race_eth_name': 'NHOPI', 'race_eth_code': 5}, {'race_eth_name': 'Other', 'race_eth_code': 8}, {'race_eth_name': 'Total', 'race_eth_code': 9}, {'race_eth_name': 'White', 'race_eth_code': 6})]
(Background on this error at: http://sqlalche.me/e/gkpj)

In [22]:
traffic_injuries.to_sql(name='traffic_injuries', con=engine, if_exists='append', index=False)

In [23]:
modes_of_transportation.to_sql(name='modes_of_transportation', con=engine, if_exists='append', index=False)

### Query tables to confirm data has been added

In [ ]:
pd.read_sql_query('select * from regions', con=engine).head()